In [ ]:
from flask import Flask, render_template, request
import requests
from flask import Flask
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import json
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
app = Flask(__name__)
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
def get_dietary_advice(text):
  url = "https://nutrition-by-api-ninjas.p.rapidapi.com/v1/nutrition"
  querystring = {"query":text}
  headers = {
    "X-RapidAPI-Key": "33ee5f19c7msh22e43b81068cb1dp18a9adjsnf084b89fcb58",
    "X-RapidAPI-Host": "nutrition-by-api-ninjas.p.rapidapi.com"
  }
  response = requests.get(url, headers=headers, params=querystring)
  data = response.json()
  return data
  pass

In [ ]:
def extract_intent(text):
    intent = {}
    # Keywords for each intent category
    type_keywords = ['cardio', 'olympic_weightlifting', 'plyometrics', 'powerlifting', 'strength', 'stretching', 'strongman']
    muscle_keywords = ["abdominals", "abductors", "adductors", "biceps", "calves", "chest", "forearms", "glutes", "hamstrings", "lats", "lower_back", "middle_back", "neck", "quadriceps", "traps", "triceps"]
    difficulty_keywords = ['beginner', 'intermediate', 'expert']
    # Extracting keywords for 'type' intent
    type_intent = [keyword for keyword in type_keywords if keyword in text]
    if type_intent:
        intent['type'] = type_intent
    # Extracting keywords for 'muscle' intent
    muscle_intent = [keyword for keyword in muscle_keywords if keyword in text]
    if muscle_intent:
        intent['muscle'] = muscle_intent
    # Extracting keywords for 'difficulty' intent
    difficulty_intent = [keyword for keyword in difficulty_keywords if keyword in text]
    if difficulty_intent:
        intent['difficulty'] = difficulty_intent
    return intent
    pass

In [ ]:
def get_workout_recommendation(text):
    querystring = extract_intent(text)
    if not querystring :
        print("Sorry, the instructions are not clear")
        return None
    url = "https://exercises-by-api-ninjas.p.rapidapi.com/v1/exercises"
    headers = {
        "X-RapidAPI-Key": "33ee5f19c7msh22e43b81068cb1dp18a9adjsnf084b89fcb58",
        "X-RapidAPI-Host": "exercises-by-api-ninjas.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    return data[0]

In [ ]:
def generate_response(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    dapi_response = str(get_dietary_advice(text))
    Wapi_response=''
    if not get_dietary_advice(text):
      Wapi_response = str(get_workout_recommendation(text))
    response = generated_text + " " + Wapi_response + ' ' + dapi_response
    return str(response)


In [ ]:

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    text = request.args.get('msg')
    response = generate_response(text)
    return response

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
